In [ ]:
import pandas as pd
import glob
import os

def load_split_csv(file_prefix, folder='data'):
    # Ishem vse kuski po prefiksu (naprimer, 'movie_part_')
    files = sorted(glob.glob(os.path.join(folder, f"{file_prefix}*")))
    # Skleivaem ih v odin DataFrame
    return pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Primer ispol'zovaniya:
df_movie = load_split_csv('movie_part_', folder='data')
df_people = load_split_csv('people_temp_part_', folder='data')
df_tmdb = load_split_csv('tmdb_full_part_', folder='notebooks')

print(f"Zagruzheno strok: {len(df_movie)}")

In [ ]:

df = pd.read_csv('../data/movie.csv')
df.shape


(218148, 9)

In [2]:
#recommandation par GENRES
#Préparer les GENRES pour la recommandation
df['genres'].head()


0                        ['Comedy']
1              ['Drama', 'Romance']
2      ['Drama', 'Comedy', 'Crime']
3    ['Drama', 'Comedy', 'Romance']
4               ['Crime', 'Comedy']
Name: genres, dtype: object

In [3]:
# transformer ce texte en vraie liste
#mettre tout en minuscule
#préparer une version “texte” pour calculer la similarité
import ast

df['genres_list'] = df['genres'].apply(ast.literal_eval)  #convertir le texte en liste
df['genres_list'].head()


0                    [Comedy]
1            [Drama, Romance]
2      [Drama, Comedy, Crime]
3    [Drama, Comedy, Romance]
4             [Crime, Comedy]
Name: genres_list, dtype: object

In [ ]:
#df['genres_list'] = df['genres_list'].apply(
  #  lambda genres: [g.lower() for g in genres]
#)

#df['genres_list'].head()


0                    [comedy]
1            [drama, romance]
2      [drama, comedy, crime]
3    [drama, comedy, romance]
4             [crime, comedy]
Name: genres_list, dtype: object

In [ ]:
#préparer une version “texte” pour calculer la similarité
df['genres_text'] = df['genres_list'].apply(lambda x: ' '.join(x))
df[['title', 'genres_text']].head()


,title,genres_text
0,Blondie,comedy
1,Love at Twenty,drama romance
2,Ariel,drama comedy crime
3,Shadows in Paradise,drama comedy romance
4,Four Rooms,crime comedy
